In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
url_1 = 'https://turbo.az'
pages_1 = '/autos?page='
tails_1 = 'q%5Bsort%5D=&q%5Bmake%5D%5B%5D=8&q%5Bmodel%5D%5B%5D=&q%5Bused%5D=&q%5Bregion%5D%5B%5D=&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bcurrency%5D=azn&q%5Bloan%5D=0&q%5Bbarter%5D=0&q%5Bcategory%5D%5B%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=&q%5Bcolor%5D%5B%5D=&q%5Bfuel_type%5D%5B%5D=&q%5Bgear%5D%5B%5D=&q%5Btransmission%5D%5B%5D=&q%5Bengine_volume_from%5D=&q%5Bengine_volume_to%5D=&q%5Bpower_from%5D=&q%5Bpower_to%5D=&q%5Bmileage_from%5D=&q%5Bmileage_to%5D=&q%5Bonly_shops%5D=&q%5Bprior_owners_count%5D%5B%5D=&q%5Bseats_count%5D%5B%5D=&q%5Bmarket%5D%5B%5D=&q%5Bcrashed%5D=1&q%5Bpainted%5D=1&q%5Bfor_spare_parts%5D='
banes = []

for j in range(1,115):
    res = requests.get(url_1 + pages_1 + str(j) + tails_1)
    content = res.text
    soup = BeautifulSoup(content, 'lxml')

    class_colour = soup.find_all('a', class_="products-i__link")
    for i in class_colour:
        url_2 = 'https://turbo.az'
        tails_2 = i['href']

        res = requests.get(url_2  + tails_2)
        content = res.text
        soup = BeautifulSoup(content, 'lxml')

        class_headers = soup.find_all('label', class_='product-properties__i-name')
        class_features = soup.find_all('span', class_='product-properties__i-value')
        class_price = soup.find('div', class_='product-price__i product-price__i--bold')
        class_number = soup.find('a', class_='product-phones__list-i')

        class_saler_name_1 = soup.find('div', class_='product-shop__owner-name')
        if class_saler_name_1 and class_saler_name_1.text.strip(): 
            saler_name = class_saler_name_1.text.strip()
        else:
            class_saler_name_2 = soup.find('div', class_='product-owner__info-name')
            saler_name = class_saler_name_2.text.strip() if class_saler_name_2 else 'değer yok'

        features_dict = {}
        
        for header, value in zip(class_headers, class_features):
            features_dict[header.text.strip()] = value.text.strip()

        features_list = []
        name_list =[]
        desired_keys = [
            'Şəhər',
            'Marka',
            'Model',
            'Buraxılış ili',
            'Ban növü',
            'Rəng',
            'Mühərrik',
            'Yürüş',
            'Sürətlər qutusu',
            'Ötürücü',
            'Yeni',
            'Yerlərin sayı',
            'Sahiblər',
            'Vəziyyəti',
            'Hansı bazar üçün yığılıb']

        for key in desired_keys:
            features_list.append(features_dict.get(key, 'değer yok'))

        features_list.append(class_price.text.strip())  # Append the price to list
        features_list.append(url_2+tails_2)  # Append the product link to list
        features_list.append(class_number.text.strip())  # Append the saler number to list
        features_list.append(saler_name)  # Append the saler name to list

        print(features_list)

        banes.append(features_list)

In [ ]:
headers = ["City", "Brand", "Model", "Year", "Body Type", "Color", "Engine Details", "Mileage", "Transmission", "Drive Type", "İs_New?", "Seat Count", "Owner Count", "Condition", "Origin",'Price','Product_link','Contact_number','Saler_name']
df = pd.DataFrame(banes, columns=headers)
